In [1]:
import pandas as pd

data = pd.read_csv("test_data.csv")

data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 3 columns):
ID        9 non-null int64
date      9 non-null object
status    9 non-null object
dtypes: int64(1), object(2)
memory usage: 296.0+ bytes


In [ ]:
df=spark.read.csv("/user/ashish/datafile.csv", header=True, dateFormat = 'YYYY-MM-DD')
df.head()

#register the a temp view 
df.createOrReplaceTempView("data")

spark.catalog.listTables()    

#execute sql over dataframe table
open_df=spark.sql(""" with open as (select ID , status, min(date) as date 
                      from data 
                      where data.status = 'open'
                      group by 1,2),   
                      
                      close as (select ID, status, max(date) as date 
                      from data 
                      where data.status = 'close'
                      group by 1,2)
                      
                      select open.ID, open.date as open_date, close.date as close_date , datediff(close.date , open.date) as days_open, datediff(close.date , open.date) * 0.5 as fees 
                      from open join close on open.ID=close.ID
                      
                      """).show()